This notebook was runned considering preprocessing of removing punctuation

In [1]:
import os 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import gensim

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset, DataLoader

# from torchtext import data

from helpers import *
from data import create_csv_submission

In [2]:
DATA_PATH = '../twitter-datasets/'
MODEL_PATH = '../models/'

# TRAIN_NEG_PATH = os.path.join(DATA_PATH, 'train_neg.txt') # 100'000 negative tweets
# TRAIN_POS_PATH = os.path.join(DATA_PATH, 'train_pos.txt') # 100'000 positive tweets
TRAIN_NEG_PATH = os.path.join(DATA_PATH, 'train_neg_full.txt') # 2'500'000 negative tweets
TRAIN_POS_PATH = os.path.join(DATA_PATH, 'train_pos_full.txt') # 2'500'000 positive tweets
TEST_PATH = os.path.join(DATA_PATH, 'test_data.txt')

In [3]:
x_text_train, y_train_full = load_data_and_labels(TRAIN_POS_PATH, TRAIN_NEG_PATH)
x_text_test = load_test_data(TEST_PATH)

In [4]:
x_text_train[0]

['i',
 'dunno',
 'justin',
 'read',
 'my',
 'mention',
 'or',
 'not',
 'only',
 'justin',
 'and',
 'god',
 'knows',
 'about',
 'that',
 'but',
 'i',
 'hope',
 'you',
 'will',
 'follow',
 'me',
 'believe',
 '15']

In [12]:
vector_length = 100
w2v_model = gensim.models.Word2Vec(x_text_train + x_text_test, 
                                   min_count=1,
                                   workers=4, 
                                   size=vector_length)

In [13]:
print(w2v_model)

Word2Vec(vocab=500249, size=100, alpha=0.025)


In [14]:
w2v_model.save(MODEL_PATH + 'twitter_w2v_v3.bin')

In [15]:
# w2v_model = gensim.models.Word2Vec.load(MODEL_PATH + 'twitter_w2v_v2.bin')

In [16]:
# When training finished delete the training model but retain the word vectors:
word_vectors = w2v_model.wv
del w2v_model

In [17]:
word_vectors.most_similar('computer')

[('internet', 0.7185307741165161),
 ('phone', 0.6644313931465149),
 ('laptop', 0.6619795560836792),
 ('ipods', 0.643019437789917),
 ('desktop', 0.6374680995941162),
 ('phones', 0.6287487745285034),
 ('browser', 0.6278807520866394),
 ('keyboard', 0.6198748350143433),
 ('wifi', 0.6100819110870361),
 ('data', 0.601195752620697)]

In [18]:
sequence_length_train = max(len(x) for x in x_text_train)
sequence_length_test = max(len(x) for x in x_text_test)
sequence_length = max(sequence_length_train, sequence_length_test)
print('Maximum sequence length of train and test data:', sequence_length)

x_text_train_pad = pad_sentences(x_text_train, padding_word="<PAD/>", sequence_length=sequence_length)
x_text_test_pad = pad_sentences(x_text_test, padding_word="<PAD/>", sequence_length=sequence_length)

del x_text_train
del x_text_test

Maximum sequence length of train and test data: 68


In [19]:
class ListDataset(Dataset):
    """Dataset wrapping data and target lists.

    Each sample will be retrieved by indexing both lists along the first
    dimension.

    Arguments:
        data_list (python list): contains sample data.
        target_list (python list): contains sample targets (labels).
    """

    def __init__(self, data_list, target_list):
        assert len(data_list) == len(target_list)
        self.data_list = data_list
        self.target_list = target_list

    def __getitem__(self, index):
        return self.data_list[index], self.target_list[index]

    def __len__(self):
        return len(self.data_list)

In [20]:
x_train, x_val, y_train, y_val = train_test_split(x_text_train_pad, 
                                                  y_train_full, 
                                                  test_size=0.1, 
                                                  random_state=42)

In [21]:
train_dataset = ListDataset(x_train, y_train)
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

In [22]:
# Network hyperparameters
N = len(train_loader.dataset.data_list)      # Number of tweets (eg 200000)
S = len(train_loader.dataset.data_list[0])   # Number of words in one sentence (eg 50)
V = vector_length                            # Length of word vectors (eg 100)
K = 3                                        # Kernel width (K*V)
C = 128                                      # Number of convolutional filters
F = 2                                        # Number of output neurons in fully connected layer

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(             # input shape (1, S, V)
                in_channels=1,              # input channels
                out_channels=C,             # number of filters
                kernel_size=(K,V),          # filter size
                padding=(1,0)               # to keep size S
        )                                   # output shape (C, S, 1)
        self.relu = nn.ReLU()               # ReLU activation
        self.max_pool1 = nn.MaxPool1d(2)    # max-pool each filter into S/2 output   
        self.conv2 = nn.Conv1d(             # input shape (C, S/2)
                in_channels=C,              # input channels
                out_channels=C,             # number of filters (one for each input channel)
                kernel_size=K,              # filter size
                padding=1                   # to keep size S/2
        )                                   # output shape (C, S/2)
        self.max_pool2 = nn.MaxPool1d(int(S/2)) # max pool each filter into 1 output
        self.out = nn.Linear(C, F)          # fully connected layer, output F classes

    def forward(self, x):
        #print(x.data.shape) #torch.Size([100, 74, 100])
        
        out = x.unsqueeze(1)
        #print(out.data.shape) #torch.Size([100, 1, 74, 100])
        
        out = self.conv1(out)
        #print(out.data.shape) #torch.Size([100, 128, 74, 1])
        
        out = self.relu(out).squeeze(3)
        #print(out.data.shape) #torch.Size([100, 128, 74])
        
        out = self.max_pool1(out)
        #print(out.data.shape) #torch.Size([100, 128, 37])
        
        out = self.conv2(out)
        #print(out.data.shape) #torch.Size([100, 128, 37])
        
        out = self.relu(out)
        #print(out.data.shape) #torch.Size([100, 128, 37])
        
        out = self.max_pool2(out).squeeze(2).float()
        #print(out.data.shape) #torch.Size([100, 128])
        
        out = self.out(out)
        #print(out.data.shape) #torch.Size([100, 2])
        
        return out

In [23]:
cnn = CNN()
print(cnn) 

CNN (
  (conv1): Conv2d(1, 128, kernel_size=(3, 100), stride=(1, 1), padding=(1, 0))
  (relu): ReLU ()
  (max_pool1): MaxPool1d (size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (max_pool2): MaxPool1d (size=34, stride=34, padding=0, dilation=1, ceil_mode=False)
  (out): Linear (128 -> 2)
)


In [24]:
num_epochs = 1
batch_size = 100
learning_rate = 0.01

optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                                  # the target label is not one-hotted

In [25]:
# TODO: SLOW!

def get_tweets_tensor(tweets, indices=[], verbose=False):
    '''Mapping every word to a vector from word2vec
    Padding words are mapped to zero
    Leave indices empty to map every tweet in tweets
    '''

    nb_tweets = len(tweets) if len(indices)==0 else len(indices)
    tweets_vec = np.zeros((nb_tweets, len(tweets[0]), vector_length), dtype=np.float32)
    
    if indices == []:
        for idx_t, tweet in enumerate(tweets):
            for idx_w, word in enumerate(tweet):
                if word != '<PAD/>':
                    tweets_vec[idx_t, idx_w] = word_vectors.wv[word]  
            if verbose and idx_t % 100000 == 0:
                print('Transformed {}/{} tweets'.format(idx_t+1, (len(x_text_train_pad))))
    else:
        for idx_t, orig_idx in enumerate(indices):
            for idx_w, word in enumerate(tweets[orig_idx]):
                if word != '<PAD/>':
                    tweets_vec[idx_t, idx_w] = word_vectors.wv[word]  
            if verbose and idx_t % 100000 == 0:
                print('Transformed {}/{} tweets'.format(idx_t+1, (len(x_text_train_pad))))
    
    return torch.from_numpy(tweets_vec)

In [26]:
for epoch in range(num_epochs):  # loop over the dataset multiple times 
    for i, batch_indices in enumerate(train_loader.batch_sampler):   # iterate over mini-batches  
        # Converting tweets to vectors and storing it in a variable
        sentences = get_tweets_tensor(train_loader.dataset.data_list, batch_indices, verbose=False)
        x = Variable(sentences)
        
        # Converting labels to a variable
        labels = torch.from_numpy(train_loader.dataset.target_list[batch_indices])
        y = Variable(labels, requires_grad=False)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad() # reset gradient
        outputs = cnn(x) # cnn output
        loss = loss_func(outputs, y) # clear gradients for this training step
        loss.backward() # backpropagation, compute gradients
        optimizer.step() # apply gradients
        
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                  %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

print('Finished Training')

Epoch [1/1], Iter [100/22500] Loss: 0.4713
Epoch [1/1], Iter [200/22500] Loss: 0.4093
Epoch [1/1], Iter [300/22500] Loss: 0.5766
Epoch [1/1], Iter [400/22500] Loss: 0.4592
Epoch [1/1], Iter [500/22500] Loss: 0.4779
Epoch [1/1], Iter [600/22500] Loss: 0.4617
Epoch [1/1], Iter [700/22500] Loss: 0.5986
Epoch [1/1], Iter [800/22500] Loss: 0.4527
Epoch [1/1], Iter [900/22500] Loss: 0.3888
Epoch [1/1], Iter [1000/22500] Loss: 0.4881
Epoch [1/1], Iter [1100/22500] Loss: 0.4845
Epoch [1/1], Iter [1200/22500] Loss: 0.4204
Epoch [1/1], Iter [1300/22500] Loss: 0.5057
Epoch [1/1], Iter [1400/22500] Loss: 0.4396
Epoch [1/1], Iter [1500/22500] Loss: 0.4381
Epoch [1/1], Iter [1600/22500] Loss: 0.3539
Epoch [1/1], Iter [1700/22500] Loss: 0.4085
Epoch [1/1], Iter [1800/22500] Loss: 0.5394
Epoch [1/1], Iter [1900/22500] Loss: 0.3993
Epoch [1/1], Iter [2000/22500] Loss: 0.5486
Epoch [1/1], Iter [2100/22500] Loss: 0.5300
Epoch [1/1], Iter [2200/22500] Loss: 0.4767
Epoch [1/1], Iter [2300/22500] Loss: 0.38

Epoch [1/1], Iter [18600/22500] Loss: 0.4168
Epoch [1/1], Iter [18700/22500] Loss: 0.4409
Epoch [1/1], Iter [18800/22500] Loss: 0.4184
Epoch [1/1], Iter [18900/22500] Loss: 0.5372
Epoch [1/1], Iter [19000/22500] Loss: 0.4272
Epoch [1/1], Iter [19100/22500] Loss: 0.5154
Epoch [1/1], Iter [19200/22500] Loss: 0.3382
Epoch [1/1], Iter [19300/22500] Loss: 0.3994
Epoch [1/1], Iter [19400/22500] Loss: 0.4104
Epoch [1/1], Iter [19500/22500] Loss: 0.5500
Epoch [1/1], Iter [19600/22500] Loss: 0.4074
Epoch [1/1], Iter [19700/22500] Loss: 0.4549
Epoch [1/1], Iter [19800/22500] Loss: 0.4309
Epoch [1/1], Iter [19900/22500] Loss: 0.4599
Epoch [1/1], Iter [20000/22500] Loss: 0.3776
Epoch [1/1], Iter [20100/22500] Loss: 0.3404
Epoch [1/1], Iter [20200/22500] Loss: 0.3822
Epoch [1/1], Iter [20300/22500] Loss: 0.3626
Epoch [1/1], Iter [20400/22500] Loss: 0.3703
Epoch [1/1], Iter [20500/22500] Loss: 0.3396
Epoch [1/1], Iter [20600/22500] Loss: 0.4932
Epoch [1/1], Iter [20700/22500] Loss: 0.4253
Epoch [1/1

In [27]:
# Evaluate n predictions from test data
n = 10000
val_output_n = cnn(Variable(get_tweets_tensor(x_val, indices=np.arange(n), verbose=False)))
y_val_pred_n = torch.max(val_output_n, 1)[1].data.numpy().squeeze()
accuracy(y_val_pred_n, y_val[:n], verbose=True)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


Accuracy: 0.8085%


0.8085

In [ ]:
test_output = cnn(Variable(get_tweets_tensor(x_text_test_pad, verbose=False)))
y_pred = torch.max(test_output, 1)[1].data.numpy().squeeze()

In [ ]:
y_pred[y_pred == 0] = -1
y_pred

In [ ]:
ids = np.arange(len(y_pred)+1)[1:]
ids

In [ ]:
create_csv_submission(ids, y_pred, 'kaggle_submission.csv')